# ConvNet Weights Initialization

In this notebook, we will create and save locally the weights of a simple Convolutional Neural Network (ConvNet) designed for the MNIST handwritten digit dataset.

### Dataset Overview

The MNIST dataset is a benchmark dataset consistsing of:

- **Features**:  
  $28 * 28$ grayscale images of handwritten digits, with $60,000$ training samples and $10,000$ test samples.
    
- **Labels**:  
  The target variable representing the digit in the image, which can take one of the following values:  
  $ \{0, 1, 2, 3, 4, 5, 6, 7, 8, 9\} $.
    

The dataset is publicly available and can be downloaded from the [MNIST dataset page](http://yann.lecun.com/exdb/mnist/) or other sources. In this notebook we will pull it using the `torchvision` library.

### ConvNet Architecture

The ConvNet is designed to be as simple as possible, ensuring efficiency and compatibility with hardware synthesis using **Vitis HLS**. The architecture includes:

1. **Convolutional Layers**:
   - **Conv1**: $32$ filters of size $3 \times 3$, stride $1$, and padding $1$.
   - **Conv2**: $64$ filters of size $3 \times 3$, stride $1$, and padding $1$.
   - Both layers use the **ReLU** activation function.

2. **Pooling Layer**:
   - Max pooling layer with a kernel size of $2 \times 2$ and stride $2$, reducing the spatial dimensions by half.

3. **Fully Connected Layers**:
   - **FC1**: Fully connected layer with $128$ neurons, using the **ReLU** activation function.
   - **Output Layer (FC2)**: Fully connected layer with $10$ neurons, corresponding to the ten digit classes (0–9). This layer does not apply an activation function, as it is followed by the softmax during training or inference.

---

Let's start by downloading the dataset using `torchvision`

In [1]:
import torchvision

dataset = torchvision.datasets.MNIST(
    "./data",
    train=True,
    download=True,
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5,), (0.5,))
    ])
)

test_dataset = torchvision.datasets.MNIST(
    "./data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5,), (0.5,))
    ])
)

We can now proceed by defining the ConvNet class, which will inherit from PyTorch's `nn.Module`.

In [2]:
import torch
import torch.nn as nn

# check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # x = [1, 28, 28]
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) # 32 filters of size 3x3. Formula for output size is: (W−F+2P)/S+1
        # x = [32, 28, 28]        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # x = [64, 28, 28]
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # pooling formula for output size is: (W−F)/S+1
        # x = [64, 14, 14]
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        # x = [128]
        self.fc2 = nn.Linear(128, 10)
        # x = [10]

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Using device: cuda


In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, random_split


# hyperparameters
batch_size = 64
NUM_EPOCHS = 10
learning_rate = 0.001

# split dataset into training and validation sets (70% train, 30% val)
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")

# initialize network, loss function and optimizer
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()

    train_accuracy = 100. * train_correct / train_total

    # validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # disable gradient calculations
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_accuracy = 100. * val_correct / val_total

    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}], "
          f"Train Loss: {train_loss / len(train_loader):.4f}, Train Acc: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Acc: {val_accuracy:.2f}%")

# test phase
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

test_accuracy = 100. * test_correct / test_total
print(f"Final Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

Training samples: 42000, Validation samples: 18000, Test samples: 10000
Epoch [1/10], Train Loss: 0.1673, Train Acc: 94.90%, Val Loss: 0.0657, Val Acc: 98.13%
Epoch [2/10], Train Loss: 0.0484, Train Acc: 98.54%, Val Loss: 0.0539, Val Acc: 98.34%
Epoch [3/10], Train Loss: 0.0288, Train Acc: 99.09%, Val Loss: 0.0485, Val Acc: 98.61%
Epoch [4/10], Train Loss: 0.0196, Train Acc: 99.35%, Val Loss: 0.0543, Val Acc: 98.50%
Epoch [5/10], Train Loss: 0.0122, Train Acc: 99.59%, Val Loss: 0.0564, Val Acc: 98.56%
Epoch [6/10], Train Loss: 0.0115, Train Acc: 99.60%, Val Loss: 0.0525, Val Acc: 98.65%
Epoch [7/10], Train Loss: 0.0098, Train Acc: 99.66%, Val Loss: 0.0544, Val Acc: 98.69%
Epoch [8/10], Train Loss: 0.0087, Train Acc: 99.72%, Val Loss: 0.0569, Val Acc: 98.81%
Epoch [9/10], Train Loss: 0.0078, Train Acc: 99.74%, Val Loss: 0.0538, Val Acc: 98.81%
Epoch [10/10], Train Loss: 0.0052, Train Acc: 99.82%, Val Loss: 0.0531, Val Acc: 98.87%
Final Test Loss: 0.0412, Test Accuracy: 99.01%


In [5]:
weights = {}
for name, param in model.to("cpu").named_parameters():
    weights[name] = param.detach().numpy()

# Print their shapes to verify the network architecture
for name, weight in weights.items():
    print(f"{name}: {weight.shape}")

conv1.weight: (32, 1, 3, 3)
conv1.bias: (32,)
conv2.weight: (64, 32, 3, 3)
conv2.bias: (64,)
fc1.weight: (128, 12544)
fc1.bias: (128,)
fc2.weight: (10, 128)
fc2.bias: (10,)


In [7]:
import numpy as np

with open('./convnet_weights.txt', 'w') as f:
    for name, weight in weights.items():
        f.write(f"{name}\n")
        if weight.ndim > 2:
            weight = weight.reshape(weight.shape[0], -1)
        np.savetxt(f, weight, fmt='%f')